In [1]:
# Module Importations
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

In [2]:
# Project Module Importations
from src.data import load_data

In [5]:
# Load dataset.
original_dataset = load_data.load_motor_data()

print(original_dataset)

ambient   coolant       u_d       u_q  motor_speed    torque  \
0      -0.752143 -1.118446  0.327935 -1.297858    -1.222428 -0.250182   
1      -0.771263 -1.117021  0.329665 -1.297686    -1.222429 -0.249133   
2      -0.782892 -1.116681  0.332771 -1.301822    -1.222428 -0.249431   
3      -0.780935 -1.116764  0.333700 -1.301852    -1.222430 -0.248636   
4      -0.774043 -1.116775  0.335206 -1.303118    -1.222429 -0.248701   
...          ...       ...       ...       ...          ...       ...   
998065 -0.047497  0.341638  0.331475 -1.246114    -1.222428 -0.255640   
998066 -0.048839  0.320022  0.331701 -1.250655    -1.222437 -0.255640   
998067 -0.042350  0.307415  0.330946 -1.246852    -1.222430 -0.255640   
998068 -0.039433  0.302082  0.330987 -1.249505    -1.222432 -0.255640   
998069 -0.043803  0.312666  0.330830 -1.246591    -1.222431 -0.255640   

             i_d       i_q        pm  stator_yoke  stator_tooth  \
0       1.029572 -0.245860 -2.522071    -1.831422     -2.066143  

In [ ]:
def split_train_test(data, test_ratio):
    """Split Training & Test Data
    ======================================
    Splits original dataset into training and evaluation data.
    
    Args:
        data (dataframe) - Original test data.
        test_ratio (int) - Ratio for splitting dataset as training percentage.
        
    Returns:
        data_train (dataframe) - Dataframe with training data slice.
        data_test (dataframe) - Dataframe with testing data slice.
    """

    # Random seed setting ensures identical data split between calls
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))

    test_set_size = int(len(data) * test_ratio)

    # Create slices of test and training indices
    train_indices = shuffled_indices[test_set_size:]
    test_indices = shuffled_indices[:test_set_size]

    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
# Split data into training and test sets
training_set, test_set = split_train_test(original_dataset, 0.8)

In [ ]:
# Drop profile id data column
training_set = training_set.drop("profile_id", axis = 1)
print(training_set.describe())

In [ ]:
# Create rotor target
rotor_training_data = training_set.drop("pm", axis = 1)
rotor_label_data = training_set["pm"].copy()

In [ ]:
# Linear Regression Model (Target - Rotor Temperature)
lin_reg = LinearRegression()
lin_reg.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(lin_reg, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(lin_reg, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "Linear Regression Model"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Standard Deviation:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Standard Deviation:", scores_mae.std())

In [ ]:
# Decision Tree Regressor Model (Target - Rotor Temperature)
tree_regr = DecisionTreeRegressor()
tree_regr.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(tree_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(tree_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "Decision Tree Regressor"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Standard Deviation:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Standard Deviation:", scores_mae.std())

In [ ]:
# Random Forest Regressor Model (Target - Rotor Temperature)
forest_regr = RandomForestRegressor()
forest_regr.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(forest_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(forest_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "Random Forest Regressor"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Standard Deviation:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Standard Deviation:", scores_mae.std())